In [84]:
#importing modules
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.feature_selection import RFE
%matplotlib inline


In [85]:
#read data
df= pd.read_csv('/Users/pieterdujardin/Desktop/1e Master Hir  /2e Sem /Advanced Analytics/assignments/2./telco_train .csv', na_values=['?'])


# How many records and attributes are there?
print('There are ' + str(df.shape[0]) + ' records')
print('There are ' + str(df.columns.size) + ' attributes:')
data_types = df.dtypes
print(data_types)


There are 5000 records
There are 38 attributes:
ID                                   int64
CHURN                                int64
START_DATE                          object
PREPAID                              int64
FIN_STATE                          float64
COUNT_PAYMENT_DELAYS_CURRENT         int64
COUNT_PAYMENT_DELAYS_1YEAR           int64
DAYS_PAYMENT_DELAYS_CURRENT          int64
DAYS_PAYMENT_DELAYS_1YEAR            int64
COMPLAINT_1WEEK                      int64
COMPLAINT_2WEEKS                     int64
COMPLAINT_1MONTH                     int64
COMPLAINT_3MONTHS                    int64
COMPLAINT_6MONTHS                    int64
CLV                                float64
COUNT_OFFNET_CALLS_1WEEK             int64
COUNT_ONNET_CALLS_1WEEK              int64
AVG_DATA_3MONTH                    float64
COUNT_CONNECTIONS_3MONTH           float64
AVG_DATA_1MONTH                    float64
COUNT_SMS_INC_ONNET_6MONTH           int64
COUNT_SMS_OUT_OFFNET_6MONTH          int64
COUNT_

In [97]:
#we conduct the exploration,cleaning,transforming,.. operations on a seperate dataframe
df_pre=df.copy()

#change following variables to categorical variables
df_pre['CHURN'] = df_pre.CHURN.astype('category')
df_pre['PREPAID'] = df_pre.PREPAID.astype('category')
df_pre['FIN_STATE']=df_pre.FIN_STATE.astype('category')

df_pre.dtypes


ID                                    int64
CHURN                              category
START_DATE                           object
PREPAID                            category
FIN_STATE                          category
COUNT_PAYMENT_DELAYS_CURRENT          int64
COUNT_PAYMENT_DELAYS_1YEAR            int64
DAYS_PAYMENT_DELAYS_CURRENT           int64
DAYS_PAYMENT_DELAYS_1YEAR             int64
COMPLAINT_1WEEK                       int64
COMPLAINT_2WEEKS                      int64
COMPLAINT_1MONTH                      int64
COMPLAINT_3MONTHS                     int64
COMPLAINT_6MONTHS                     int64
CLV                                 float64
COUNT_OFFNET_CALLS_1WEEK              int64
COUNT_ONNET_CALLS_1WEEK               int64
AVG_DATA_3MONTH                     float64
COUNT_CONNECTIONS_3MONTH            float64
AVG_DATA_1MONTH                     float64
COUNT_SMS_INC_ONNET_6MONTH            int64
COUNT_SMS_OUT_OFFNET_6MONTH           int64
COUNT_SMS_INC_OFFNET_1MONTH     

In [87]:
from datetime import datetime
date_format = "%Y-%m-%d"

b = datetime.strptime('2013-10-31', date_format)

for i in range(df_pre.shape[0]):
    a = datetime.strptime(str(df_pre.loc[df_pre.index[i],'START_DATE']), date_format)
    delta= b - a
    df_pre.loc[df_pre.index[i],'START_DATE']= delta.days

                          
df_pre['START_DATE']=df_pre.START_DATE.astype(int)
df_pre.rename(columns={'START_DATE':'DAYS_SUBSCRIBED'}, inplace=True)
df_pre.head()


,ID,CHURN,DAYS_SUBSCRIBED,PREPAID,FIN_STATE,COUNT_PAYMENT_DELAYS_CURRENT,COUNT_PAYMENT_DELAYS_1YEAR,DAYS_PAYMENT_DELAYS_CURRENT,DAYS_PAYMENT_DELAYS_1YEAR,COMPLAINT_1WEEK,...,COUNT_SMS_OUT_ONNET_1MONTH,COUNT_SMS_OUT_ONNET_WKD_1MONTH,AVG_MINUTES_INC_OFFNET_1MONTH,AVG_MINUTES_INC_ONNET_1MONTH,MINUTES_INC_OFFNET_WKD_1MONTH,MINUTES_INC_ONNET_WKD_1MONTH,AVG_MINUTES_OUT_OFFNET_1MONTH,AVG_MINUTES_OUT_ONNET_1MONTH,MINUTES_OUT_OFFNET_WKD_1MONTH,MINUTES_OUT_ONNET_WKD_1MONTH
0,11452,0,983,0,NaN,0,4,0,8,0,...,12,1,20.893333,11.131667,30.050000,23.050000,77.783333,10.553333,78.166667,21.933333
1,16546,0,103,0,NaN,0,0,0,0,0,...,32,5,41.351667,20.648333,11.816667,79.166667,66.683333,186.300000,33.666667,23.833333
2,15607,0,185,0,NaN,0,2,0,47,0,...,54,6,129.450000,64.466667,100.100000,18.733333,27.450000,59.566667,16.916667,13.816667
3,13621,0,434,1,NaN,0,4,0,51,0,...,0,0,96.783333,11.308333,11.550000,44.666667,19.600000,10.966667,15.216667,39.966667
4,11710,0,864,0,NaN,0,3,0,11,0,...,32,5,24.593333,32.368333,88.400000,73.233333,10.593333,229.650000,28.133333,0.000000


In [88]:
df_pre = df_pre.drop(columns= ['ID', 'FIN_STATE', 'AVG_DATA_3MONTH', 'COUNT_CONNECTIONS_3MONTH' , 'AVG_DATA_1MONTH'])                   

In [89]:
df_pre.head()

,CHURN,DAYS_SUBSCRIBED,PREPAID,COUNT_PAYMENT_DELAYS_CURRENT,COUNT_PAYMENT_DELAYS_1YEAR,DAYS_PAYMENT_DELAYS_CURRENT,DAYS_PAYMENT_DELAYS_1YEAR,COMPLAINT_1WEEK,COMPLAINT_2WEEKS,COMPLAINT_1MONTH,...,COUNT_SMS_OUT_ONNET_1MONTH,COUNT_SMS_OUT_ONNET_WKD_1MONTH,AVG_MINUTES_INC_OFFNET_1MONTH,AVG_MINUTES_INC_ONNET_1MONTH,MINUTES_INC_OFFNET_WKD_1MONTH,MINUTES_INC_ONNET_WKD_1MONTH,AVG_MINUTES_OUT_OFFNET_1MONTH,AVG_MINUTES_OUT_ONNET_1MONTH,MINUTES_OUT_OFFNET_WKD_1MONTH,MINUTES_OUT_ONNET_WKD_1MONTH
0,0,983,0,0,4,0,8,0,0,0,...,12,1,20.893333,11.131667,30.050000,23.050000,77.783333,10.553333,78.166667,21.933333
1,0,103,0,0,0,0,0,0,0,0,...,32,5,41.351667,20.648333,11.816667,79.166667,66.683333,186.300000,33.666667,23.833333
2,0,185,0,0,2,0,47,0,0,0,...,54,6,129.450000,64.466667,100.100000,18.733333,27.450000,59.566667,16.916667,13.816667
3,0,434,1,0,4,0,51,0,0,0,...,0,0,96.783333,11.308333,11.550000,44.666667,19.600000,10.966667,15.216667,39.966667
4,0,864,0,0,3,0,11,0,0,0,...,32,5,24.593333,32.368333,88.400000,73.233333,10.593333,229.650000,28.133333,0.000000


In [90]:
df_pre.isnull().sum()
df_pre.isnull().values.any()

False

In [91]:
df_split= df_pre.copy()

X=df_split.drop(columns='CHURN')
y=df_split['CHURN']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


X_train.head()

,DAYS_SUBSCRIBED,PREPAID,COUNT_PAYMENT_DELAYS_CURRENT,COUNT_PAYMENT_DELAYS_1YEAR,DAYS_PAYMENT_DELAYS_CURRENT,DAYS_PAYMENT_DELAYS_1YEAR,COMPLAINT_1WEEK,COMPLAINT_2WEEKS,COMPLAINT_1MONTH,COMPLAINT_3MONTHS,...,COUNT_SMS_OUT_ONNET_1MONTH,COUNT_SMS_OUT_ONNET_WKD_1MONTH,AVG_MINUTES_INC_OFFNET_1MONTH,AVG_MINUTES_INC_ONNET_1MONTH,MINUTES_INC_OFFNET_WKD_1MONTH,MINUTES_INC_ONNET_WKD_1MONTH,AVG_MINUTES_OUT_OFFNET_1MONTH,AVG_MINUTES_OUT_ONNET_1MONTH,MINUTES_OUT_OFFNET_WKD_1MONTH,MINUTES_OUT_ONNET_WKD_1MONTH
1334,328,0,0,0,0,0,0,0,0,0,...,12,6,12.416667,5.100000,43.500000,0.516667,207.700000,48.666667,37.183333,0.216667
4768,128,0,0,0,0,0,0,0,0,0,...,0,0,25.333333,11.400000,26.866667,0.716667,93.333333,0.000000,39.433333,10.083333
65,239,0,0,0,0,0,0,0,0,0,...,0,0,31.250000,0.000000,50.166667,0.000000,154.800000,24.333333,120.200000,11.100000
177,739,0,0,1,0,10,0,0,0,0,...,2,1,117.866667,30.866667,0.000000,7.966667,38.700000,23.716667,0.433333,5.933333
4489,480,0,0,2,0,12,1,1,1,1,...,12,8,29.488333,480.350000,44.650000,17.548333,290.200000,47.256667,56.566667,175.400000


In [92]:
logmodel=LogisticRegression()
logmodel.fit(X_train,y_train)

/Users/pieterdujardin/anaconda3/envs/tryout1/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [93]:
predictions = logmodel.predict(X_test)


In [98]:
from sklearn.metrics import classification_report
classification_report(y_test,predictions)

'              precision    recall  f1-score   support\n\n           0       0.81      0.94      0.87      1059\n           1       0.77      0.46      0.57       441\n\n   micro avg       0.80      0.80      0.80      1500\n   macro avg       0.79      0.70      0.72      1500\nweighted avg       0.79      0.80      0.78      1500\n'

In [94]:
X_train.head()
y_test.head()

2764    0
4767    0
3814    1
3499    0
2735    0
Name: CHURN, dtype: category
Categories (2, int64): [0, 1]

In [95]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt 
logit_roc_auc = roc_auc_score(y_test, logmodel.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, logmodel.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

TypeError: object of type 'CategoricalDtype' has no len()

5000